In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline

/home/code_dilip/AI_and_ML/Machine-Learning/Deep_Learning/Neural_Networks_and_deep_Learning/Week 2/Logistic Regression as a Neural Network


In [2]:
import os
os.getcwd() 

'/home/code_dilip/AI_and_ML/Machine-Learning/Deep_Learning/Neural_Networks_and_deep_Learning/Week 2/Logistic Regression as a Neural Network'

In [4]:
def load_dataset():
    train_dataset = h5py.File('/home/code_dilip/AI_and_ML/Machine_Learning_intro/Deeplearning.ai_Neural_networks/aulas_keras/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('/home/code_dilip/AI_and_ML/Machine_Learning_intro/Deeplearning.ai_Neural_networks/aulas_keras/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/code_dilip/AI_and_ML/Machine_Learning_intro/Deeplearning.ai_Neural_networks/aulas_keras/train_catvnoncat.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/code_dilip/AI_and_ML/Machine_Learning_intro/Deeplearning.ai_Neural_networks/aulas_keras/train_catvnoncat.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

We added "_orig" at the end of image datasets (train and test) because we are going to preprocess them. After preprocessing, we will end up with train_set_x and test_set_x (the labels train_set_y and test_set_y don't need any preprocessing).

Each line of your train_set_x_orig and test_set_x_orig is an array representing an image. You can visualize an example by running the following code. Feel free also to change the `index` value and re-run to see other images. 

In [6]:
# Example of a picture
index =10
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[0, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

NameError: name 'train_set_x_orig' is not defined

In [6]:
### START CODE HERE ### (≈ 3 lines of code)
m_train = train_set_x_orig.shape[0]
m_test =  test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]
### END CODE HERE ###

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

Number of training examples: m_train = 209
Number of testing examples: m_test = 50
Height/Width of each image: num_px = 64
Each image is of size: (64, 64, 3)
train_set_x shape: (209, 64, 64, 3)
train_set_y shape: (1, 209)
test_set_x shape: (50, 64, 64, 3)
test_set_y shape: (1, 50)


In [7]:
X_train = train_set_x_orig
Y_train = train_set_y
X_test = test_set_x_orig
Y_test = test_set_y

In [8]:
print("X_train.shape:",X_train.shape)
print("Y_train.shape:",Y_train.shape)
print()
print("X_test.shape:",X_test.shape)
print("Y_test.shape:",Y_test.shape)

X_train.shape: (209, 64, 64, 3)
Y_train.shape: (1, 209)

X_test.shape: (50, 64, 64, 3)
Y_test.shape: (1, 50)


In [9]:
X_train_flatten = X_train.reshape(X_train.shape[1]*X_train.shape[2]*X_train.shape[3],X_train.shape[0])
X_test_flatten = X_test.reshape(X_test.shape[1]*X_test.shape[2]*X_test.shape[3],X_test.shape[0])
print("X_train_flatten.shape:",X_train_flatten.shape)
print('X_test_flatten.shape:',X_test_flatten.shape)

X_train_flatten.shape: (12288, 209)
X_test_flatten.shape: (12288, 50)


In [10]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    ### START CODE HERE ### (≈ 1 line of code)
    s = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return s

In [11]:
def initialize_with_zeros(dim):

    W = np.zeros((dim,1))
    b = 0
    return (W,b)

In [12]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

w = [[0.]
 [0.]]
b = 0


In [13]:
def propagate(W,b,X,Y):

    m = X.shape[1]
    Z = np.dot(W.T,X)+b
    A = sigmoid(Z)

    cost = (-1/m)*(np.dot(Y,np.log(A+1).T) + np.dot((1-Y),np.log((1-A)+1).T))
    dW = (np.dot(X,(A-Y).T))/m
    db = np.sum(A-Y)/m
    gradients = {'dW':dW,'db':db}

    return (gradients,cost)

In [14]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w,b,X,Y)
print ("dw = " + str(grads["dW"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601]
 [2.39507239]]
db = 0.001455578136784208
cost = [[-0.23252594]]


**Expected Output**:

<table style="width:50%">
    <tr>
        <td>  ** dw **  </td>
      <td> [[ 0.99845601]
     [ 2.39507239]]</td>
    </tr>
    <tr>
        <td>  ** db **  </td>
        <td> 0.00145557813678 </td>
    </tr>
    <tr>
        <td>  ** cost **  </td>
        <td> 5.801545319394553 </td>
    </tr>

</table>

In [15]:
def optimize(W,b,X,Y,num_iterations,learning_rate,print_cost):

    costs = []
    for i in range(1,num_iterations+1):
        
        (grads,cost) = propagate(W,b,X,Y)
        dw = grads['dW']
        db = grads['db']
        W = W - (learning_rate*dw)
        b = b - (learning_rate*db)
        costs.append(cost)

        if(i%100==0):
            print("{}/{} iteration!".format(i,num_iterations))
            if(print_cost):
                print("cost:",cost)
                
        params = {'W':W,'b':b}
    return (params,grads,costs)

In [16]:
(params,grads,costs) = optimize(w,b,X,Y,num_iterations=100,learning_rate=0.009,print_cost = False)

print ("w = " + str(params["W"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dW"]))
print ("db = " + str(grads["db"]))

100/100 iteration!
w = [[0.19033591]
 [0.12259159]]
b = 1.9253598300845747
dw = [[0.67752042]
 [1.41625495]]
db = 0.21919450454067654


**Expected Output**: 
<table style="width:40%">
    <tr>
       <td> **w** </td>
       <td>[[ 0.19033591] [ 0.12259159]] </td>
    </tr>
    <tr>
       <td> **b** </td>
       <td> 1.92535983008 </td>
    </tr>
    <tr>
       <td> **dw** </td>
       <td> [[ 0.67752042]
 [ 1.41625495]] </td>
    </tr>
    <tr>
       <td> **db** </td>
       <td> 0.219194504541 </td>
    </tr>
</table>

In [17]:
def predict(W,b,X):

    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    Z = np.dot(W.T,X)+b
    A = sigmoid(Z)
    Y_prediction = (A>=0.5)*1.0

    return Y_prediction

In [18]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[1. 1. 0.]]


In [19]:
def model(X_train,Y_train,X_test,Y_test,num_iterations,learning_rate,print_cost):

    (W,b) = initialize_with_zeros(X_train.shape[0])
    (parameters,grads,costs) = optimize(W,b,X_train,Y_train,num_iterations,learning_rate,print_cost)
    (W,b) = (parameters['W'],parameters['b'])
    Y_train_predict = predict(W,b,X_train)
    Y_test_predict = predict(W,b,X_test)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_train_predict - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_test_predict - Y_test)) * 100))

    d = {'costs':costs,
         'dW':grads['dW'],
         'db':grads['db'],
         'Y_prediction_train':Y_train_predict,
         'Y_prediction_test':Y_test_predict,
         'learning_rate':learning_rate,
         'num_iterations':num_iterations}

    return d

In [21]:
def z_score_normalization(x):
    mu = np.mean(x,axis=0)
    sigma = np.std(x,axis=0)
    x_norm = (x - mu)/sigma
    return (mu,sigma,x_norm)

In [23]:
(mu_tr,sigma_tr,X_train_norm) = z_score_normalization(X_train_flatten)
(mu_t,sigma_t,X_test_norm) = z_score_normalization(X_test_flatten)
print("x_train_norm.shape:",X_train_norm.shape)
print("x_test_norm.shape:",X_test_norm.shape)

x_train_norm.shape: (12288, 209)
x_test_norm.shape: (12288, 50)


In [26]:
d = model(X_train_norm, Y_train, X_test_norm, Y_test, num_iterations = 2000, learning_rate = 0.0099, print_cost = True)

100/2000 iteration!
cost: [[-0.37601324]]
200/2000 iteration!
cost: [[-0.48892864]]
300/2000 iteration!
cost: [[-0.51183728]]
400/2000 iteration!
cost: [[-0.54949121]]
500/2000 iteration!
cost: [[-0.64403684]]
600/2000 iteration!
cost: [[-0.66657023]]
700/2000 iteration!
cost: [[-0.67285342]]
800/2000 iteration!
cost: [[-0.67572757]]
900/2000 iteration!
cost: [[-0.67751477]]
1000/2000 iteration!
cost: [[-0.67880435]]
1100/2000 iteration!
cost: [[-0.67981224]]
1200/2000 iteration!
cost: [[-0.68063823]]
1300/2000 iteration!
cost: [[-0.68133644]]
1400/2000 iteration!
cost: [[-0.68193967]]
1500/2000 iteration!
cost: [[-0.68246943]]
1600/2000 iteration!
cost: [[-0.68294061]]
1700/2000 iteration!
cost: [[-0.68336397]]
1800/2000 iteration!
cost: [[-0.68374755]]
1900/2000 iteration!
cost: [[-0.68409752]]
2000/2000 iteration!
cost: [[-0.68441872]]
train accuracy: 100.0 %
test accuracy: 52.0 %
